# SelectGrid

In [ ]:
import pandas as pd, traitlets as T, ipywidgets as W
from ipywxyz import SelectGrid, DockBox

In [ ]:
flex = dict(layout=dict(flex="1"))

In [ ]:
def make_select_grid(GridClass=SelectGrid, df=None):
    pg = DockBox(layout=dict(height="60vh"))
    dg = pg.dg = GridClass()
    dg2 = pg.dg2 = GridClass()
    dg3 = pg.dg3 = GridClass()
    
    rando = None
    if df is not None:
        dg.value = df
    else:
        with W.Output():
            @W.interact
            def rando(rows=(1, 1000), cols=(1, 1000)):
                dg.value = pd.np.random.rand(rows, cols)
    x, y = [
        [
            W.jslink((w, "value"), (dg, f"scroll_{v}")),
            W.jslink((pg.dg, f"max_{v}"), (w, "max")),
            w,
        ][-1]
        for i, v in enumerate("xy")
        for w in [W.IntSlider(description=f"${v}$", layout=dict(flex="1"))] 
    ]
    
    def _slice(c):
        s = dg.selection
        dg2.value = dg.value.iloc[s[2]: s[3]+ 1, s[0]:s[1] + 1]
    
    dg.observe(_slice, ["selection", "value"])
    
    def _slice2(c):
        s = dg2.selection
        dg3.value = dg2.value.iloc[s[2]: s[3]+ 1, s[0]:s[1] + 1]
    
    dg2.observe(_slice2, ["selection", "value"])
    
    pg.children = [
        W.VBox([*(rando.widget.children if rando else []), x, y]), 
        dg, 
        dg2,
        dg3
    ]
    dg.description = "dataset"
    dg2.description = "subset"
    dg3.description = "sub-subset"
    pg.hide_tabs = True
    @pg.on_displayed
    def on_display(*args, **kwargs):
        pg.dock_layout = {'type': 'split-area',
 'orientation': 'horizontal',
 'children': [{'type': 'tab-area', 'widgets': [1], 'currentIndex': 0},
  {'type': 'tab-area', 'widgets': [2], 'currentIndex': 0},
  {'type': 'split-area',
   'orientation': 'vertical',
   'children': [{'type': 'tab-area', 'widgets': [3], 'currentIndex': 0},
    {'type': 'tab-area', 'widgets': [0], 'currentIndex': 0}],
   'sizes': [0.6, 0.4]}],
 'sizes': [0.41894974454222295, 0.24771692212444382, 0.3333333333333333]}
    return pg

In [ ]:
if __name__ == "__main__":
    pg = make_select_grid()
    display(pg)